# Master Discogs Database

In [46]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap

### Master DB code
from masterdb import masterdb
from mainDB import mainDB


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-06-15 17:49:32.193518


In [47]:
mdb = myMusicDBMap(debug=True)

Creating myMusicDBMap()
   Loading my music db map: /anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Counter({'Discogs': 3924, 'AllMusic': 3887, 'MusicBrainz': 3674, 'LastFM': 2775, 'RockCorner': 466, 'DatPiff': 458, 'AceBootlegs': 175, 'CDandLP': 80, 'RateYourMusic': 30, 'MusicStack': 5, 'MetalStorm': 0})


In [61]:
maindb = mainDB(mdb=mdb, create=False)
#maindb.setDBFull() ## Do this to recreate everything
maindb.setDBKnown()
artistDBs = maindb.getKnownArtistDBs()

Creating DB Info For Discogs
Creating DB Info For AllMusic
Creating DB Info For MusicBrainz
Creating DB Info For AceBootlegs
Creating DB Info For RateYourMusic
Creating DB Info For LastFM
Creating DB Info For DatPiff
Creating DB Info For RockCorner
Creating DB Info For CDandLP
Creating DB Info For MusicStack
Creating DB Info For MetalStorm


,ID,Name,Ref,Artist,Num,DiscArtist
Discogs,144998,Black Sabbath,/artist/144998-Black-Sabbath,Black Sabbath,NaN,Black Sabbath
AllMusic,0000771438,Black Sabbath,/artist/black-sabbath-mn0000771438/discography,Black Sabbath,None,Black Sabbath
MusicBrainz,133638912340787458619495685266666501922,Black Sabbath,/artist/5182c1d9-c7d2-4dad-afa0-ccfeada921a8,Black Sabbath,None,Black Sabbath
AceBootlegs,760604981735,Black Sabbath,[https://ace-bootlegs.com/category/black-sabba...,Black Sabbath,None,Black Sabbath
LastFM,8092241712,Black Sabbath,https://www.last.fm/music/Black%2BSabbath/+alb...,Black Sabbath,NaN,Black Sabbath
DatPiff,2241712,Black Sabbath,None,Black Sabbath,NaN,Black Sabbath
RockCorner,92241712,Black Sabbath,http://www.therockcorner.com/black-sabbath/,Black Sabbath,None,Black Sabbath


In [4]:
disc.getArtistIDToRefData()

NameError: name 'disc' is not defined

In [5]:
from pandas import Series, DataFrame
disc = maindb.getDBData("Discogs")["Disc"]
artistIDtoName  = Series(disc.getArtistIDToNameData())
artistIDtoRef   = Series(disc.getArtistIDToRefData())
if False:    
        print("Creating Pandas DataFrame for {0} Artists".format(artistIDtoName.shape[0]))
        cols = ["Name"]
        discdf = DataFrame(artistIDtoName)
        discdf.columns = cols
        print("\tShape --> {0}".format(discdf.shape))
        
        discdf["Ref"] = artistIDtoRef

        print("  Finding Real Artist Name")
        discdf[["Artist", "Num"]] = DataFrame(discdf['Name'].apply(self.realName).tolist(), index=discdf.index)
        print("\tShape --> {0}".format(discdf.shape))

        print("  Removing None Artist")
        #discdf = discdf[~discdf["Artist"].isna()]
        print("\tShape --> {0}".format(discdf.shape))

        print("  Finding Disc Artist Name")
        discdf["DiscArtist"] = discdf['Artist'].apply(self.discConv)
        print("\tShape --> {0}".format(discdf.shape))

        if self.disc.base == "musicbrainz":
            print("  Cleaning Disc Artist Name (MusicBrainz Only)")
            discdf["DiscArtist"] = discdf['DiscArtist'].apply(self.cleanMB)
            print("\tShape --> {0}".format(discdf.shape))

In [9]:
cols = ["Name"]
discdf = DataFrame(artistIDtoName)
discdf.columns = cols
discdf


,Name,Ref
1000,Dave Clarke,/artist/1000-Dave-Clarke
1000500,Club Pulse,/artist/1000500-Club-Pulse
100200,Dike,/artist/100200-Dike
1002000,Larry Stokes,/artist/1002000-Larry-Stokes
1002600,Gidd Sanchez,/artist/1002600-Gidd-Sanchez
1002900,John Keding,/artist/1002900-John-Keding
1003900,Jason Pancho,/artist/1003900-Jason-Pancho
100500,Liviiing,/artist/100500-Liviiing
1005400,Gapeseed,/artist/1005400-Gapeseed
1009400,Polish Radio Symphony Orchestra,/artist/1009400-Polish-Radio-Symphony-Orchestra


In [ ]:
%load_ext autoreload
%autoreload

dbs = dbdata.keys()
dbs = ["DatPiff", "RockCorner", "CDandLP", "MusicStack", "MetalStorm"]
dbs = ["Discogs", "AllMusic", "MusicBrainz", "LastFM"]
dbs = ["MusicBrainz"]
#dbs = ["DatPiff", "RockCorner", "CDandLP", "MusicStack", "MetalStorm"]

########################################################################################################
## Loop over DBs
########################################################################################################
for db in dbs:
    print("\n")
    print("="*125)
    print("="*125)
    print("=",db)
    print("="*125)
    print("="*125)
    print("\n")
    
    
    disc  = dbdata[db]["Disc"]
    mymdb = masterdb(db, disc, force=True)
    mymdb.setMyMusicDB(mdb)
    
    
    #################################
    # Artists
    #################################
    mymdb.createArtistIDMap()
    slimArtistDB      = mymdb.getSlimArtistDB()
    print("All   --> {0}".format(slimArtistDB.shape))
    knownSlimArtistDB = mymdb.getKnownSlimArtistDB()
    print("Known --> {0}".format(knownSlimArtistDB.shape))
        
    
    #################################
    # Artist Albums
    #################################
    mymdb.createArtistAlbumIDMap()
    slimArtistAlbumsDB      = mymdb.getSlimArtistAlbumsDB()
    print("All Albums    --> {0}".format(slimArtistAlbumsDB.shape))
    knownSlimArtistAlbumsDB = mymdb.getKnownSlimArtistAlbumsDB()
    print("Known Albums  --> {0}".format(knownSlimArtistAlbumsDB.shape))

In [ ]:
mdb = myMusicDBMap(debug=False)
musicmap = mdb.get()

In [ ]:
knownSlimArtistDB

In [ ]:

    
    

########################################################################################################################
#
# Artist DB
#
########################################################################################################################
def getArtistDB(disc, force=False):
    start, cmt = clock("\n=================================== Creating Artist DB ===================================")
    if force is False:
        print("Using previously created Artist DB")
        discdf = disc.getMasterArtistDiscogsDB()
        elapsed(start, cmt)
        return discdf
    
    print("Loading ArtistID Data")
    artistIDtoName  = Series(disc.getArtistIDToNameData())
    artistIDtoRef   = Series(disc.getArtistIDToRefData())
    artistIDToVariations  = Series(disc.getArtistIDToVariationsData())

    print("Creating Pandas DataFrame for {0} Artists".format(artistIDtoName.shape[0]))
    cols = ["Name"]
    discdf = DataFrame(artistIDtoName)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Ref")
    discdf = discdf.join(DataFrame(artistIDtoRef))
    cols += ["Ref"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Variations")
    discdf = discdf.join(DataFrame(artistIDToVariations))
    cols += ["Variations"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    discdf["Known"] = True
    
    print("  Finding Real Artist Name")
    discdf[["Artist", "Num"]] = DataFrame(discdf['Name'].apply(self.realName).tolist(), index=discdf.index)
    print("\tShape --> {0}".format(discdf.shape))

    

    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)

    print("Saving Master Artist DB File")
    saveFilename = disc.getMasterArtistDiscogsDBFilename()
    saveFile(ifile=saveFilename, idata=discdf, debug=False)
    
    return discdf    
    
    
    

########################################################################################################################
#
# Artist Metadata DB
#
########################################################################################################################
def getArtistMetadataDB(disc, force=True):
    start, cmt = clock("\n=================================== Creating Artist Metadata DB ===================================")
    if force is False:
        print("Using previously created Artist Metadata DB")
        discdf = disc.getMasterArtistMetadataDiscogsDB()
        elapsed(start, cmt)
        return discdf
    
    print("Loading ArtistID Data")
    artistIDtoGenre          = Series(disc.getArtistIDToGenreData())
    artistIDtoStyle          = Series(disc.getArtistIDToStyleData())
    artistIDToCollaboration  = Series(disc.getArtistIDToCollaborationData())

    print("Creating Pandas DataFrame for {0} Artists".format(artistIDtoGenre.shape[0]))
    cols = ["Genre"]
    discdf = DataFrame(artistIDtoGenre)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Style")
    discdf = discdf.join(DataFrame(artistIDtoStyle))
    cols += ["Style"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Collaboration")
    discdf = discdf.join(DataFrame(artistIDToCollaboration))
    cols += ["Collaboration"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)

    print("Saving Master Artist Metadata DB File")
    saveFilename = disc.getMasterArtistMetadataDiscogsDBFilename()
    saveFile(ifile=saveFilename, idata=discdf, debug=False)
    
    return discdf




########################################################################################################################
#
# Artist Albums DB
#
########################################################################################################################
def getArtistAlbumsDB(disc, loadRefs=False, force=False):
    start, cmt = clock("\n=================================== Creating Artist Albums DB ===================================")
    if force is False:
        print("Using previously created Artist Albums DB")
        discdf = disc.getMasterArtistAlbumsDiscogsDB()
        elapsed(start, cmt)
        return discdf
    
    print("Loading ArtistID Data")
    artistIDtoAlbumNames  = Series(disc.getArtistIDToAlbumNamesData())
    if loadRefs:
        artistIDtoAlbumRefs   = Series(disc.getArtistIDToAlbumRefsData())

    print("Creating Pandas DataFrame for {0} Artists".format(artistIDtoAlbumNames.shape[0]))
    cols = ["Albums"]
    discdf = DataFrame(artistIDtoAlbumNames)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("DataFrame Shape is {0}".format(discdf.shape))
    
    print("Saving Master Artist Albums DB File")
    saveFilename = disc.getMasterArtistAlbumsDiscogsDBFilename()
    saveFile(ifile=saveFilename, idata=discdf, debug=False)
    
    elapsed(start, cmt)
        
    return discdf






########################################################################################################################
#
# Artist Album Known DB
#
########################################################################################################################
def getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB):
    start, cmt = clock("\n=================================== Creating Artist Album DB ===================================")
    from pandas import Series, DataFrame
    
    idx=discAlbumDB.index
    
    tmpdb = discArtistAlbumsDB["Albums"].copy()
    print("Creating Pandas DataFrame for {0} Arist Albums".format(tmpdb.shape[0]))
    discdf = DataFrame(tmpdb.apply(isKnownAlbum, idx=idx).tolist(), index=tmpdb.index)
    discdf.columns = ["Known Albums", "All Albums", "Albums"]
    print("\tShape --> {0}".format(discdf.shape))
    
    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)
    
    return discdf
    
def isKnownAlbum(x, **kwargs):
    retval = {}
    albumSummary = [0, 0]
    for mediaType in x.keys():
        for albumID in x[mediaType].keys():
            albumName = x[mediaType][albumID]
            #print(mediaType,albumID,albumName,'\t\t',end="")
            known     = albumID in kwargs['idx']
            #print(known)
            
            retval[albumID] = [albumName, mediaType, known]
            albumSummary[0] += known
            albumSummary[1] += 1
            
    return [albumSummary[0], albumSummary[1], retval]





########################################################################################################################
#
# Album DB
#
########################################################################################################################
def getAlbumDB(disc):
    start, cmt = clock("\n=================================== Creating Artist Album DB ===================================")
    from pandas import Series, DataFrame
    print("Loading AlbumID Data")
    albumIDtoName    = Series(disc.getAlbumIDToNameData())
    albumIDtoRef     = Series(disc.getAlbumIDToRefData())
    albumIDToArtists = Series(disc.getAlbumIDToArtistsData())

    print("Creating Pandas DataFrame for {0} Albums".format(albumIDtoName.shape[0]))
    cols = ["Name"]
    discdf = DataFrame(albumIDtoName)
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Ref")
    discdf = discdf.join(DataFrame(albumIDtoRef))
    cols += ["Ref"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("  Joining Artists")
    discdf = discdf.join(DataFrame(albumIDToArtists))
    cols += ["Artists"]
    discdf.columns = cols
    print("\tShape --> {0}".format(discdf.shape))

    print("DataFrame Shape is {0}".format(discdf.shape))
    elapsed(start, cmt)
    
    return discdf





########################################################################################################################
#
# Master DB Join
#
########################################################################################################################
def createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB):
    start, cmt = clock("\n=================================== Creating Artist ID DB ===================================")
    print("Creating Pandas DataFrame for {0} Arist IDs".format(discArtistDB.shape[0]))
    print("  Joining Artist Metadata")
    discdf = discArtistDB.join(discArtistMetadataDB)
    print("\tShape --> {0}".format(discdf.shape))
    print("  Joining Artist Albums")
    discdf = discdf.join(discArtistAlbumKnownDB)
    print("\tShape --> {0}".format(discdf.shape))
    elapsed(start, cmt)

    savename = disc.getMasterDiscogsDBFilename()
    saveFile(idata=discdf, ifile=savename, debug=True)
    
    
    
    
    
    
########################################################################################################################
#
# Associated Functions
#
########################################################################################################################
def directoryName(x):
    if x is None:
        return x
    if "..." in x:
        x = x.replace("...", "")
    if "/" in x:
        x = x.replace("/", "-")
    return x

def realName(x):
    if x is None:
        return [None,-1]
    
    lenx = len(x)
    if len(x) < 1:
        return [x,-1]

    if x[-1] != ")":
        return [x, None]
    

    if lenx >=5:
        if x[-3] == "(":
            try:
                num = int(x[-2:-1])
                val = x[:-3].strip()
                return [val, num]
            except:
                return [x, None]
            
    if lenx >= 6:
        if x[-4] == "(":
            try:
                num = int(x[-3:-1])
                val = x[:-4].strip()
                return [val, num]
            except:
                return [x, None]
            
    if lenx >= 7:
        if x[-4] == "(":
            try:
                num = int(x[-3:-1])
                val = x[:-4].strip()
                return [val, num]
            except:
                return [x, None]

    return [x, None]

def discConv(x):
    if x is None:
        return ""
    x = x.replace("/", "-")
    x = x.replace("¡", "")
    while x.startswith(".") and len(x) > 1:
        x = x[1:]
    x = x.strip()
    return x

def cleanMB(x):
    pos = [x.rfind("(")+1, x.rfind(")")]
    if sum([p > 0 for p in pos]) != len(pos):
        return x
    parval = x[pos[0]:pos[1]]
    return x[:pos[0]-2].strip()

# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

In [ ]:
%load_ext autoreload
%autoreload

from lookup import createArtistIDMap
from masterdb import getSlimArtistDB
from lookup import createArtistAlbumIDMap
from masterdb import getArtistAlbumsDB

dbs = dbdata.keys()
dbs = ["AllMusic"]

########################################################################################################
## Loop over DBs
########################################################################################################
for db in dbs:
    print("\n")
    print("="*125)
    print("="*125)
    print("=",db)
    print("="*125)
    print("="*125)
    print("\n")
    
    
    ####################################################################################################
    ## Create Artist ID Lookup
    ####################################################################################################
    createArtistIDMap(dbdata[db]["Disc"])
    
    
    ####################################################################################################
    ## Create Artist ID Pandas DataFrame
    ####################################################################################################
    discSlimArtistAMDB      = getSlimArtistDB(dbdata[db]["Disc"], force=True)
    discKnownSlimArtistAMDB = getKnownSlimArtistDB(dbdata[db]["Disc"], mydb, force=True)


    ####################################################################################################
    ## Create Artist ID to Album ID Lookup
    ####################################################################################################
    createArtistAlbumIDMap(dbdata[db]["Disc"])


    ####################################################################################################
    ## Create Artist ID Pandas DataFrame
    ####################################################################################################
    discArtistAMAlbumsDB = getArtistAlbumsDB(dbdata[db]["Disc"], force=True)

In [ ]:
%load_ext autoreload
%autoreload
from myMusicDBMap import myMusicDBMap
from artistDB import artistDB

mdb = myMusicDBMap(debug=True)
mydb = mdb.get()

***
***
***

## Artist ID --> Genre, Style, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discAM)

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discDC)

## Album ID --> Name, Ref, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

In [ ]:

discArtistLMAlbumsDB = getArtistAlbumsDB(discRM, force=False)

In [ ]:
from collections import Counter
cntr = Counter()
for artistData in discArtistLMAlbumsDB["Albums"]:
    for key in artistData.keys():
        cntr[key] += 1
cntr

# Pandas DB

## Slim Artist DB

In [ ]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

## Artist DB

In [ ]:
from masterdb import getArtistDB

In [ ]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

## Artist Metadata DB

In [ ]:
from masterdb import getArtistMetadataDB

In [ ]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

## Artist Albums DB

In [ ]:
from masterdb import getArtistAlbumsDB

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=False)
discArtistAlbumsDB.head()

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

## Albums DB

In [ ]:
from masterdb import getAlbumDB

In [ ]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

## Artist Album ID --> Known Albums

In [ ]:
from masterdb import getArtistAlbumKnownDB

In [ ]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

# Joining Artist ID DataFrame

In [ ]:
from masterdb import createMasterDB

In [ ]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

In [ ]:
from random import random

# Download Artist Data

In [ ]:
artistName = "Killin' Baudelaire"

In [ ]:
dbsToGet = ["AllMusic", "Discogs", "MusicBrainz", "LastFM", "RockCorner", "CDandLP"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist, dbdata["Discogs"]["Artists"].searchDiscogForArtist,
            dbdata["MusicBrainz"]["Artists"].searchMusicBrainzForArtist, dbdata["LastFM"]["Artists"].searchLastFMForArtist,
            dbdata["RockCorner"]["Artists"].searchRockCornerForArtist, dbdata["CDandLP"]["Artists"].searchCDandLPForArtist]

dbsToGet = ["AllMusic"]
searches = [dbdata["AllMusic"]["Artists"].searchAllMusicForArtist]

In [ ]:
for i,artistName in enumerate(mdb.getArtists()):
        if artistName == "Riff Raff":
            print(i)
            break

In [ ]:
def searchAll(mdb, minI=-1):
    nArtists = len(mdb.getArtists())
    for i,artistName in enumerate(mdb.getArtists()):
        if i <= minI:
            continue

        print('\n\n\n')
        print('-'*130)
        print(i,'/',nArtists,'===>\t',artistName)
        print('-'*130)
        print('\n')
        
        dbdata = mdb.getArtistData(artistName)
        if all([dbdata.get("LastFM"), dbdata.get("Discogs"), dbdata.get("AllMusic"), dbdata.get("MusicBrainz")]):
            print("{0} is well known.".format(artistName))
            continue
        else:
            print("{0} get it!".format(artistName))

        for search in searches:
            print("====>",search)
            try:
                search(artistName)
            except:
                sleep(3)

In [ ]:
searchAll(mdb, 2383)

In [ ]:
# 3404